In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Simular datos
np.random.seed(42)
n = 500
X1 = np.random.normal(0, 1, n)
X2 = np.random.normal(2, 1.5, n)
logits = -1 + 0.8 * X1 + 1.2 * X2
prob = 1 / (1 + np.exp(-logits))
y = np.random.binomial(1, prob)

# Crear DataFrame
df = pd.DataFrame({'X1': X1, 'X2': X2, 'y': y})
X = df[['X1', 'X2']]
y = df['y']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir grid de hiperparámetros para MLPRegressor
param_grid = {
    'hidden_layer_sizes': [(5,), (10,), (5, 5)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],  # Regularización L2
    'solver': ['adam'],  # SGD también es posible
    'max_iter': [500]
}

# Inicializar modelo y GridSearchCV
mlp = MLPRegressor(random_state=42)
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train, y_train)

# Mejor modelo y predicciones
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluación con MSE
mse = mean_squared_error(y_test, y_pred)

# Imprimir resultados
print("Mejores hiperparámetros:", grid_search.best_params_)
print(f"Error cuadrático medio (MSE) en test: {mse:.4f}\n")

# Mostrar todos los resultados del grid
results = pd.DataFrame(grid_search.cv_results_)
results = results[['param_hidden_layer_sizes', 'param_activation', 'param_alpha', 'mean_test_score', 'std_test_score']]
results['mean_test_score'] = -results['mean_test_score']  # Convertimos a MSE positivo
results = results.rename(columns={'mean_test_score': 'mean_MSE', 'std_test_score': 'std_MSE'})

print("Resumen de resultados (ordenados por MSE):")
print(results.sort_values(by='mean_MSE'))


Mejores hiperparámetros: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (5, 5), 'max_iter': 500, 'solver': 'adam'}
Error cuadrático medio (MSE) en test: 0.1562

Resumen de resultados (ordenados por MSE):
   param_hidden_layer_sizes param_activation param_alpha  mean_MSE   std_MSE
11                   (5, 5)             tanh      0.0001  0.158467  0.011874
14                   (5, 5)             tanh       0.001  0.158468  0.011875
17                   (5, 5)             tanh        0.01  0.158473  0.011881
15                     (5,)             tanh        0.01  0.165074  0.014530
12                     (5,)             tanh       0.001  0.165079  0.014536
9                      (5,)             tanh      0.0001  0.165080  0.014537
6                      (5,)             relu        0.01  0.165847  0.018701
3                      (5,)             relu       0.001  0.165871  0.018740
0                      (5,)             relu      0.0001  0.165874  0.018744
4          